In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import warnings
pd.options.mode.chained_assignment = 'raise'
warnings.simplefilter("error")
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("market_info_forge.csv")

In [25]:
dfn = df.loc[df["typeID"] == 18]
dfn = dfn["avgPrice"].pct_change()
dfn = dfn[1:].fillna(method="ffill")

In [10]:
del df["Unnamed: 0"]
del df["regionID"]

In [11]:
end_dfn = pd.DataFrame()
end_dfm = pd.DataFrame()

count = 0
total_outliers = 0

with open('item_ids_selected.txt') as f: # grab all item ids that you want to investigate from a text file
    
    while True: 
        line = f.readline() # iterate over all item ids

        count += 1
        print(count / 73)        
        if line == "":
            break
        
        else:
            n = int(line)
            dfn = df.loc[df["typeID"] == n] # select seperate df by type id
            dfn = dfn.assign(date = pd.to_datetime(dfn.date)) 
            dfn = dfn.loc[(dfn["date"].dt.year != 2003) & (dfn["date"].dt.year != 2018)] # exclude the first year (highly volatile) and last year ()
            dfn = dfn.assign(retd = dfn["avgPrice"].pct_change()) # Create daily percentage changes of the avgPrice column

            for c in dfn.columns: # iterate over all columns and remove outliers based on the mean of the respective column
                if c in ["date", "typeID"]:
                    continue
                if not dfn.empty:
                    dfn.loc[dfn[c] > 5 * dfn[c].mean(), c] = np.nan # remove values greater than 5*mean of the respective column
                    total_outliers += dfn[c].isnull().sum()
                    dfn[c].fillna(method='ffill', inplace=True) # replace with values previously seen 
            
            #----------------------------------
            # Creates measures by year (std) and days (momentum) and then appends them to every corresponding year/day.
            # Process could most likely be vectorized but unsure how.
            # r100 is the accumulated return over the last 100 days of the year. (momentum effect)
            # r100yr is the accumulated return from the beginning of the year until the last 100 days. (momentum effect)
            # std is the standard deviation calculated on daily returns per year. (as per Amihud (2002) and Ben-Raphael, Kadan and Wohl (2008))
            #----------------------------------
            for y in set(dfn["date"].dt.year): # iterate through all years in the original data
                curyrdf = dfn.loc[dfn["date"].dt.year == y] # current year dataframe
                r100yrdf = dfn.loc[(dfn["date"].dt.year == y) & (dfn["date"].dt.dayofyear < 265)] # dataframe of the first 265 days of the current year
                r100df = dfn.loc[(dfn["date"].dt.year == y) & (dfn["date"].dt.dayofyear > 265)]# dataframe of the last 100 days of the current year
                
                if r100df.empty: # if no data in the last 100 days of the current year then r100 is nan
                    r100 = np.nan
                else:
                    r100 = ((r100df["avgPrice"].iloc[-1] - r100df["avgPrice"].iloc[0]) / r100df["avgPrice"].iloc[0]) # create return over last 100 days
                    
                if r100yrdf.empty: # same check as above but now for the first 265 days
                    r100yr = np.nan
                else:
                    r100yr = ((r100yrdf["avgPrice"].iloc[-1] - r100yrdf["avgPrice"].iloc[0]) / r100yrdf["avgPrice"].iloc[0])
                
                std = curyrdf["retd"].std() #create standard deviation from daily returns
                amihud = np.sum(np.absolute(curyrdf["retd"]) / (curyrdf["volume"] * curyrdf["avgPrice"])) / 365.25 # Amihud measure as per Amihud (2002)
                
                indices_curyear = dfn.loc[dfn["date"].dt.year == y].index # grab indices of this year ! COULD POSSIBLY BE VECTORIZED SO NOT OPTIMAL
                
                for i in indices_curyear: # Append the following measures for every day in the current year
                    dfn.loc[i, "stdy"] = std * 100 # *100 as per Amihud (2002)
                    dfn.loc[i, "r100yr"] = r100yr
                    dfn.loc[i, "r100"] = r100
                    dfn.loc[i, "amihud"] = amihud

            dfn = dfn.set_index("date") # set date as index    
                
            dfn = dfn.assign(ln_voli =  np.log(dfn["volume"] * dfn["avgPrice"])) # create log volume denoted in isk
            dfn = dfn.assign(cpqs = ((dfn["highPrice"] - dfn["lowPrice"]) / ((dfn["highPrice"] + dfn["lowPrice"]) / 2))) # Closing Percent Quoted Spread, best measure according to Fong, Holden and Trzcinka

            
            dfd = dfn.iloc[1:] # renaming for convience sake and removing the first entry because of NaN returns
            
            dfm = dfn.resample("M").mean() # convert daily series to monthly means
            dfm = dfm.assign(retm = dfm["avgPrice"].pct_change()) # monthly returns
            dfm = dfm.iloc[1:] # remove first month again because of NaN returns
            
            if not dfd.empty:
                end_dfn = end_dfn.append(dfd, sort = True) # create csv file of daily data
                end_dfm = end_dfm.append(dfm, sort = True) # create csv file of monthly data
        

0.0136986301369863
0.0273972602739726
0.0410958904109589


KeyboardInterrupt: 

In [5]:
print(total_outliers)

35927


In [6]:
end_dfm.to_csv("market_info_forge_monthly.csv")
end_dfn.to_csv("market_info_forge_daily.csv")